In [93]:
import geopandas as gp
import pandas as pd
import os
import json
import pylab as pl
import csv
import urllib2
import numpy as np
# import zipfile as zp
import statsmodels.api as sm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Importing Energy_and_Water_Data_Disclosure_for_Local_Law_84 Data

In [111]:
url = 'https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv?accessType=DOWNLOAD'
response = urllib2.urlopen(url)
energy = gp.GeoDataFrame.from_csv(response)

# creating simpler variable name for later
energy['BBL'] = energy['NYC Borough, Block, and Lot (BBL)']
energy['EUI'] = energy['Site EUI(kBtu/ft2)']

print energy.columns

Index([u'NYC Borough, Block, and Lot (BBL)', u'Co-reported BBL Status',
       u'BBLs Co-reported',
       u'Reported NYC Building Identificaiton Numbers (BINs)',
       u'Street Number', u'Street Name', u'Borough', u'Zip Code',
       u'DOF Benchmarking Submission Status', u'Site EUI(kBtu/ft2)',
       u'Weather Normalized Site EUI(kBtu/ft2)', u'Source EUI(kBtu/ft2)',
       u'Weather Normalized Source EUI(kBtu/ft2)',
       u'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)',
       u'Automatic Water Benchmarking Eligible', u'Reported Water Method',
       u'ENERGY STAR Score', u'Total GHG Emissions(MtCO2e)',
       u'Direct GHG Emissions(MtCO2e)', u'Indirect GHG Emissions(MtCO2e)',
       u'Reported Property Floor Area (Building(s)) (ft²)',
       u'DOF Property Floor Area (Buildngs and Parking)(ft2)',
       u'Primary Property Type - Self Selected', u'DOF Number of Buildings',
       u'BBL', u'EUI'],
      dtype='object')


### Importing PLUTO Data

In [65]:
# url2 = 'http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/mn_mappluto_15v1.zip'
# response2 = urllib2.urlopen(url2)
# #  = zp.ZipFile.open(name[, mode[, pwd]])

# zipped = zp.ZipFile(response2, 'r')
# pluto = zipped.open('Manhattan/MNMapPLUTO.shp')

In [94]:
pluto = gp.GeoDataFrame.from_file(os.getenv('PUI2015')+'/HW5/Manhattan/MNMapPLUTO.shp')
print pluto.columns

Index([    u'APPBBL',    u'APPDate',    u'Address', u'AllZoning1',
       u'AllZoning2', u'AreaSource', u'AssessLand',  u'AssessTot',
              u'BBL',   u'BldgArea',  u'BldgClass',  u'BldgDepth',
        u'BldgFront',      u'Block',   u'BoroCode',    u'Borough',
         u'BsmtCode',  u'BuiltCode',   u'BuiltFAR',     u'CB2010',
               u'CD',     u'CT2010',    u'ComArea',    u'CommFAR',
          u'CondoNo',    u'Council',  u'EDesigNum',  u'Easements',
       u'ExemptLand',  u'ExemptTot',        u'Ext',   u'FacilFAR',
       u'FactryArea',   u'FireComp', u'GarageArea', u'HealthArea',
         u'HistDist', u'IrrLotCode',    u'LandUse',   u'Landmark',
              u'Lot',    u'LotArea',   u'LotDepth',   u'LotFront',
          u'LotType',  u'LtdHeight', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'OfficeArea',  u'OtherArea',   u'Overlay1',
         u'Overlay2',  u'OwnerName',  u'OwnerType', u'PLUTOMapID',
       u'PolicePrct',   u'ProxCode',    u'ResArea',   u'ResidF

In [66]:
# from pandas.tools.plotting import scatter_matrix
# scatter_matrix (energy, s=300, figsize=(12, 12), diagonal='kde')

### Data Cleanup

In [95]:
def canconvert(mydata):
    try :
        float(mydata)
        return True
    except ValueError:
        return False
    
fun = np.vectorize(canconvert)

In [116]:
index_units = fun(pluto['UnitsTotal'])
pluto['UnitsTotal'][~index_units]=float('NaN')

index_pbbl = fun(pluto['BBL'])
pluto['BBL'][~index_pbbl]=float('NaN')

index_use = fun(energy['EUI'])
energy['EUI'][~index_use]=float('NaN')

index_ebbl = fun(energy['BBL'])
energy['BBL'][~index_ebbl]=float('NaN')

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning:

In [98]:
# print energy.EUI.astype(float)
# print energy.BBL.astype(float)
# print pluto.BBL.astype(float)
# print pluto.UnitsTotal.astype(float)

###Merging the two data sets using the BBL

In [99]:
joined = pd.merge(energy,pluto,on=['BBL'],how='inner')
len(joined.BBL)

6545

### Additional cleanup - removing outliers

In [100]:
joined = joined[joined['EUI']>0]
len(joined.BBL)

5097

###Finding linear regression model

In [127]:
eui = []
eui = [int(x) for x in joined['EUI'][:]]
type(eui[3])

ValueError: invalid literal for int() with base 10: '82.1'

In [107]:
X = sm.add_constant(np.log10(joined.UnitsTotal))
print X
model = sm.OLS(np.log10(joined.EUI), X, missing='drop').fit() # ignores entires where x or y is NaN
# fit = model.fit()
# return fit.params[1], fit.params[0], fit # could also return stderr in each via fit.bse

      const  UnitsTotal
0         1    1.380211
1         1    1.792392
2         1    1.929419
3         1    2.107210
4         1    1.000000
5         1    1.785330
6         1    2.686636
7         1    2.656098
8         1    1.000000
10        1    1.832509
11        1    1.949390
12        1    0.954243
13        1    1.579784
14        1    0.698970
15        1    1.857332
16        1    1.748188
17        1    1.792392
18        1    1.977724
19        1    1.785330
21        1    1.785330
22        1    2.720986
23        1    1.724276
24        1    1.633468
25        1    1.819544
26        1    2.290035
28        1    1.623249
30        1    2.404834
31        1    1.716003
32        1    0.845098
33        1    0.954243
...     ...         ...
5744      1    1.875061
5746      1    0.000000
5747      1    0.000000
5748      1    0.000000
5749      1        -inf
5751      1    1.000000
5755      1    0.903090
5757      1    2.096910
5760      1    1.579784
5764      1    0

AttributeError: 'str' object has no attribute 'log10'